<a href="https://colab.research.google.com/github/Ali-MH-Mansour/BoWC-Method/blob/main/Bag-of-weighted-concepts/Self_FastText_BoWC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Connect Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Requirements  || Определение библиотек**

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
!pip install sklearn
!pip install spherecluster
!pip install soyclustering
#################################### Importing all Library here  ################################################
from soyclustering import SphericalKMeans
import time
import re
import string
import pandas as pd
import numpy as np
import math
from numpy import array 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from numpy import array
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import gensim.downloader
from collections import Counter 
import functools
import math
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
#from sklearn_extra.cluster import KMedoids
#from spherecluster import SphericalKMeans
############ Our Downloads ##########
nltk.download('punkt')
nltk.download('stopwords')
##################################
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer('english')

################################ 
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize, sent_tokenize, pos_tag

##############################################################################################
def final_preprocess_text(text):
    # 1. Tokenise to alphabetic tokens
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = convert_to_lower(text)
    text = remove_white_space(text)
    text = remove_short_words(text)
    tokens = toknizing(text)

    # 2. POS tagging
    pos_map = {'J': 'a', 'N': 'n', 'R': 'r', 'V': 'v'}
    pos_tags = pos_tag(tokens)
    
    # 3. Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(t.lower(), pos=pos_map.get(p[0], 'v')) for t, p in pos_tags]
    return tokens

#--------------------------------------------------------------------------------------------

def convert_to_lower(text):
  return text.lower()
#--------------------------------------------------------------------------------------------
def remove_numbers(text):
  text = re.sub(r'\d+' , '', text)
  return text
#--------------------------------------------------------------------------------------------

def remove_short_words(text):
  text = re.sub(r'\b\w{1,2}\b', '', text)
  return text
#--------------------------------------------------------------------------------------------

def remove_punctuation(text):
     punctuations = '''!()-[]{};:'"\,<>./?@#$%^+&*_~'''
     no_punct = ""
     for char in text:
        if char not in punctuations:
            no_punct = no_punct + char
     return no_punct
#--------------------------------------------------------------------------------------------
def remove_white_space(text):
  text = text.strip()
  return text
#--------------------------------------------------------------------------------------------
def toknizing(text):
  stop_words = set(stopwords.words('english'))
  tokens = word_tokenize(text)
  ## Remove Stopwords from tokens
  result = [i for i in tokens if not i in stop_words]
  return result
#--------------------------------------------------------------------------------------------
def remove_duplication(text):
  return list(set(text)) 
####################################################################################
def pre_processing_dataset(data):
  All_tokens = [];
  for i in range(len(data)):
    All_tokens.append(final_preprocess_text(data.iloc[i][1]))
  All_tokens = Convert_list_of_list_to_list(All_tokens)
  print('the number of words in Crorpus' , len(All_tokens))

  return All_tokens

####################################################################################
def Convert_list_of_list_to_list(all_tokens):
  tokens = []
  for words in all_tokens:
    for i in words:
      tokens.append(i)
  return tokens

#########################################################################################
def Create_embedding_matrix(model , tokens , embedding_dim):
    embedding_matrix = []
    irregular_words  = []
    for words in tokens:
          try :
            temp = model.wv[words]
            embedding_matrix.append(temp)
          except KeyError:
            # for the words that are not in model vocabulary move to irregular_words
            irregular_words.append(words)
    return embedding_matrix , irregular_words

#####################################################

     |████████████████████████████████| 154 kB 5.2 MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# list of stop words from many resources
my_stopwords= ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab","aab",'aaab', "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]

In [ ]:
####################################################################################
### loading the embeddings
def Load_pre_trained_model_embeddings(file_path):
  wv = KeyedVectors.load(file_path, mmap='r')
  return wv
###########################################################################################
# K-means Clustering the embeddings #
def embeddings_k_means_clustering(embeddings_vectors, N_clusters):
  #kmedoids = KMedoids(n_clusters=5, random_state=0).fit(embeddings_vectors)
  #SphericalKMeans
  kmeans = KMeans(n_clusters= N_clusters , random_state=0).fit(embeddings_vectors)
  return kmeans
########################################################################################
def spherical_kmeans(n_clusters_, embeddings_matrix,sim_digree):
  embeddings_matrix_csr = csr_matrix(embeddings_matrix)
  spherical_kmeans = SphericalKMeans( max_similar=sim_digree, init='similar_cut', 
      n_clusters = n_clusters_)
  labels = spherical_kmeans.fit_predict(embeddings_matrix_csr)
  centers = spherical_kmeans.cluster_centers_
  return labels , centers

############################################################################################

#############################################################################################
def clusters_properties(k_means):
  print(Counter(k_means.labels_))
  centers = k_means.cluster_centers_
  '''
  Return
     - Number of samples for each cluter ,
     - The Centers of each cluster
  '''
  return Counter(k_means.labels_) , centers
#############################################################################################
def Save_words_with_Embeddings(tokens , pre_model):
  dic = {}
  for words in tokens: 
      try :
        dic.update({words : pre_model.wv[words]})
      except: 
        continue;
  return dic
##############################################################################################
### this function take the vector and return the word that linked with it.
def Vector_to_Word(vector , dic):
  word = ''
  for i in dic:
    if functools.reduce(lambda x, y : x and y, map(lambda p, q: p == q,list(dic[i]),vector), True): 
      word = i
  return word
##############################################################################################

def get_cluster_elements(labels , n_clusters , embedding_matrix):
  # Nice Pythonic way to get the indices of the points for each corresponding cluster
  mydict = {i: np.where(labels == i)[0] for i in range(n_clusters)}
  embeddings_samples = []
  for i in range(len(mydict)):
    temp= []
    for j in mydict[i]:
      temp.append(embedding_matrix[j])
    embeddings_samples.append(temp)
    del temp
  return embeddings_samples

############################################################################################
def get_soreted_dictionary(centers , dictionary , M):
  dic_with_similarity =[]
  for i in range(len(dictionary)):
    temp = []
    for j in range(len(dictionary[i])):
      temp.append([dictionary[i][j] , cosine_similarity([dictionary[i][j]] , [centers[i]])])
    dic_with_similarity.append(temp)
    del temp
      ####### Sort all vectors clusters ############
  sorted_list = []
  def take_second(elem):
      return elem[1]
  for i in range(len(dic_with_similarity)):
      sorted_list.append(sorted(dic_with_similarity[i], key=take_second , reverse = True))
    ########### Get the hightest M features that related to the its centers among whole concepet dictionary#############
  Top_M_Concept = [sorted_list[i][:M] for i in range(len(dic_with_similarity))]
  return Top_M_Concept

############################################################################################
# compare each words in documents with all clusters words.
def Get_similarity_matrix(v1 , list_of_vectors):
  sim = []
  s=0
  for i in list_of_vectors:
    sim.append(cosine_distance(v1, i[0]))
  sim = array(sim).astype('float').reshape(-1).tolist()
  if len(sim)!=0:
    s=sum(sim)/len(sim)
  return s

### convert document to Vector with size C * M 
##############################################################################################
##############################################################################################
def Doc2Vec(embeddeings_document,number_of_concept_words,alpha,centers,dictionary):
  M=2 # concept array elements
  N=5 # cluster words
  concept_freq = 0
  ## intilize doc array
  doc2vec = np.zeros((len(centers), M)).tolist()
  final_vector=np.zeros(len(centers)).tolist()
  for cnt in range(len(centers)):
    counter = 0;
    sim =[]
    NN=0
    for emb_word in embeddeings_document:
      Wsim =cosine_distance(emb_word, centers[cnt])# التشابه مع اقرب كلمة للمركز
      if Wsim  > alpha and NN<=N:
        NN=NN+1
        concept_freq = concept_freq+1 #for tf
        Smean =Get_similarity_matrix(emb_word , dictionary[cnt][0:number_of_concept_words-1]);
        sim.append(float((Smean+Wsim)/2))
        doc2vec[cnt][M-1] = doc2vec[cnt][M-1] + 1;
      elif Wsim  > alpha and NN>N:
        concept_freq = concept_freq+1 #for tf
        sim.append(Wsim)
        doc2vec[cnt][M-1] = doc2vec[cnt][M-1] + 1;
      else:
        continue;
      counter = counter +1;
    # end for doc words with one concepts
    meanofSim=0# mean of sim for current doc
    if(len(sim)!=0):
      meanofSim=sum(sim)/len(sim)
    doc2vec[cnt][M-2]=meanofSim
    del sim

  if concept_freq !=0: # the number of occurence for all concept in this document
    for i in doc2vec:
      i[M-1] = i[M-1]* math.exp(i[M-2]) / concept_freq
  
  doc2vec1=np.array(doc2vec,dtype=object)
  final_vector=doc2vec1[:,1].tolist() 

  return final_vector
###################################################################
def cosine_distance(u, v):
  return np.dot(u, v) / (math.sqrt(np.dot(u, u)) * math.sqrt(np.dot(v, v)))
#########################################33
def normalize_vector(vect):
  bymax=[float(i)/max(vect) for i in vect]
  bysum=[float(i)/sum(vect) for i in vect]
  return bymax,bysum


####################################################33333
def self_embedding_all_documents(df , model,N):
  documents_embeddings = []
  for i in range(len(df)):
    documents_embeddings.append(Self_convert_doc_to_embedding(df['document'][i] , model,N))
  return documents_embeddings

########################################################################################################
def Self_convert_doc_to_embedding(text , model,N):
  tokens = final_preprocess_text(text)
  if N!=0:
    tokens=removeElements(tokens,N)
  embeddings_matrix  , irrugular_words = Create_embedding_matrix(model , tokens , 300)
  return embeddings_matrix
#########################################################################################################


####################################################33333
def embedding_all_documents(df , model,N):
  documents_embeddings = []
  for i in range(len(df)):
    documents_embeddings.append(convert_doc_to_embedding(df['document'][i] , model,N))
  return documents_embeddings

#########################################################################################################
def BoWC_representation(documents_embeddings , sim_val , alpha , centers , dictionary):
  embeddings = []
  for i in documents_embeddings: 
    v = Doc2Vec(i ,sim_val , alpha , centers , dictionary)
    embeddings.append(v)
  return embeddings
##########################################

def calculate_IDF(doc_vectors):
  docL=len(doc_vectors[0])
  N=len(doc_vectors) #collection size
  IDf = np.zeros(docL).tolist() # بطول العناقيد
  for doc in doc_vectors:
    counter=0 # counting concepts
    for concept_freature in doc:
      if concept_freature > 0:
        IDf[counter] = IDf[counter] + 1 #  i[M-1]
      counter = counter+1
  
  #ضرب حساب التردد
  newIDF=[]
  for i in IDf:
    if i!=0:
      newIDF.append( math.exp(-1*(i/N)))
    else:
      newIDF.append(0)
  return IDf, newIDF

########################################### 
#Calculate Cf-IDF
def CF_IDF(doc_vectors,IDF):
  docL=len(doc_vectors[0])
  for i in range(len(doc_vectors)):
    for j in range(docL):
       doc_vectors[i][j] = doc_vectors[i][j] * IDF[j]
  return doc_vectors

#####################################################
def clac_new_Tf_idf(to_delete,tf_similarity,idfcount,idfExp):
  i=0
  j=0
  newIDFexp=[]
  newIDFcount=[]
  newTf_similarity=[]
  num_of_concepts=len(idfexp)

  
  for d in tf_similarity:
    tempD=[]
    for j in range(num_of_concepts):
      if j not in to_delete:
        tempD.append(d[j])
    newTf_similarity.append(tempD)
    del tempD
  ###
  for i in range(num_of_concepts):
    if i not in todelete:
      newIDFexp.append(idfexp[i])
      newIDFcount.append(idfcount[i])
  return newTf_similarity,newIDFexp,newIDFcount
#########################################################################################################

def saveList(myList,filename):
    # the filename should mention the extension 'npy'
    np.save(filename,myList)
    print("Saved successfully!")
def loadList(filename):
    # the filename should mention the extension 'npy'
    tempNumpyArray=np.load(filename)
    return tempNumpyArray.tolist()

#######################################################
# number of doc in each class 
def DataBalance(data):
  data['class'] = pd.factorize(data['class'])[0]
  uniqueValues = data['class'].unique()
  l=len(uniqueValues)
  labels_freq = np.zeros(l).tolist()
  for i in range(len(data)):
    label= data.iloc[i][0]
    labels_freq[label]=labels_freq[label]+1
  return labels_freq 

############################################
from collections import Counter 
def removeElements(lst, k): 
    counted = Counter(lst) 
    temp_lst = [] 
    for el in counted: 
        if counted[el] < k: 
            temp_lst.append(el) 
    res_lst = [] 
    for el in lst: 
        if el not in temp_lst: 
            res_lst.append(el) 
              
    return(res_lst)
################################################################################
#thershold for words in document
def delete_empty_docs(documents_embeddings,labels,thershold):
  doc_indexes=[]
  new_emb=[]
  new_label=[]
  j=0
  #find empty docs
  for d in documents_embeddings:
    l=len(d)
    if l>thershold:
      new_label.append(labels[j])
      new_emb.append(documents_embeddings[j])
    else:
      doc_indexes.append(j)
    j=j+1
  print(doc_indexes)
  return new_emb,new_label
#################################
#used for self embeddings 
def get_documents_as_list(df):
  documents_embeddings = []
  for i in range(len(df)):
    documents_embeddings.append(final_preprocess_text(df['document'][i]))
  return documents_embeddings

def documents_as_list_of_sentences(df):
  documents_embeddings = []
  for i in range(len(df)):
    tokens = final_preprocess_text(df['document'][i])
    sentens= ' '.join(tokens)
    documents_embeddings.append(sentens)
  return documents_embeddings
#######################################
def word_count(documents_embeddings):
  word_count=[]
  #find empty docs
  for d in documents_embeddings:
    l=len(d)
    word_count.append(l)
    
  return word_count
##########################################
def document_biggerThan(data_DF, doc_vectors,th): 
  #number of docs in each class
  doc_in_class=DataBalance(data_DF)
  # number of classes
  L_labels=len(doc_in_class)
  K_clusters=len(doc_vectors[0])
  #array for freq  
  freq_array = np.zeros(L_labels).tolist()
  mean_array = np.zeros(L_labels).tolist()  
  i=0 # doc counter
  for d in doc_vectors:
    curren_class=data_DF['class'][i]
    docL= len(d)
    if docL>th:
      freq_array[curren_class]=freq_array[curren_class]+1
    i=i+1
  #متوسط اطوال مستندات كل فئة=عدد مستنداتها الاكبر من حد معين على عدد مستندات الفئة
  mean_array = [i / j for i, j in zip(freq_array, doc_in_class)] 

  return freq_array,mean_array

##############################################################
def concept_to_delete(min_idf,max_idf,idfcount):
  to_delete_List=[]
  j=0
  for i in idfcount: #30
    if (i > max_idf) or (i < min_idf):
      to_delete_List.append(j)
    j+=1
  return to_delete_List


#############################################
def data_details(documents_embeddings,data_DF,th):
  fr,men=document_biggerThan(data_DF,documents_embeddings,th)
  no_docs = len(data_DF)
  doc_per_class = DataBalance(data_DF)
  wc=word_count(documents_embeddings) #embedded word counts
  o_wc= docs_word_count(data_DF) # orginal word count

  print('№ of all doc=',len(data_DF),"\n")
  print("Befor embeddings: \n",
        'the mean length of docs', sum(o_wc)/len(o_wc), "\n",
        "The max length of docs", max(o_wc),"\n",
        "The min length of docs", min(o_wc),"\n", 
        '№ classes ', ' =', len(doc_per_class),"\n",
        "№ docs per class ", doc_per_class )
  print("After embeddings: \n",
        "the mean length of embeded docs", sum(wc)/len(wc), "\n",
        "The max length of embeded docs", max(wc),"\n",
        "The min length of embeded docs", min(wc),"\n", 
        "№ Documents with length > ",th, "=" ,sum(fr))
##################################################################

# Загрузка Датасетов (Data sets Downolad from Drive)
ВВС & 20NG & Reuters & OHSUMED & WebKB

In [ ]:
Reuters_file_path="/content/drive/MyDrive/PHD Work/Data/Reuters88.csv"

data_DF = pd.read_csv(Reuters_file_path)

# rename columns
data_DF.columns.values[0] = "class"
data_DF.columns.values[1] = "document"

# Factorize class names to numbers
data_DF['class'] = pd.factorize(data_DF['class'])[0]
data_DF.head()

,class,document
0,0,ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...
1,1,CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...
2,2,JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...
3,3,THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...
4,1,SRI LANKA GETS USDA APPROVAL FOR WHEAT PRICE\n...


###**Предварительная обработка**

## load or extract tokens from dataset

In [ ]:
# extract token from dataset
#all_tokens = pre_processing_dataset(data_DF)
token_path="/content/drive/MyDrive/PHD Work/PreProData/Reuters_r20_data.txt.npy"

all_tokens=loadList(token_path)


## Select self or pretrained embedding model

In [ ]:
# Train fast text model on your dataset
documents_as_list = get_documents_as_list(data_DF)
from gensim.models import FastText
model = FastText(documents_as_list, size=300, window=15, min_count=5, workers=4,sg=1)

# Or Load pre trained model embeddings
#model = Load_pre_trained_model_embeddings(model_path)

## Extract Dictionary of words

In [ ]:
# Remove duplication and get the unique words 
unique_words = remove_duplication(all_tokens)
print("unique words: " ,len(unique_words),"\n")

# make dictionary {token : emnbedding vector} for 
dic = Save_words_with_Embeddings (unique_words , model)
print("length of dictionary: ",len(dic),"\n")

unique words:  3174 

length of dictionary:  3174 



## Clustering the Dictionary of words

In [ ]:
dim = 300 #model.vector_size

# Save (word:embedding) for each word in each document
embeddings_matrix  , irrugular_words = Create_embedding_matrix(model , unique_words , dim)

In [ ]:
# number of clusters
K_clusters= 400
sim_digree=0.3
number_of_words_per_cluster=5

######### Clustering
labels , centers = spherical_kmeans(K_clusters , embeddings_matrix, sim_digree)
print("The clusters details", Counter(labels), "\n")

# dictionary cluster index: values are its words embedding
concept_dictionary = get_cluster_elements(labels, K_clusters , embeddings_matrix)

start_time = time.time()
sorted_cons = get_soreted_dictionary(centers , concept_dictionary , number_of_words_per_cluster)
print("Clustering excution time --- %s seconds ---" % (time.time() - start_time), "\n")

The clusters details Counter({398: 41, 399: 31, 393: 28, 314: 27, 6: 26, 301: 26, 227: 23, 23: 23, 302: 23, 118: 22, 61: 22, 129: 22, 174: 22, 37: 21, 368: 21, 386: 21, 364: 21, 53: 20, 11: 20, 45: 20, 245: 20, 333: 20, 176: 20, 291: 20, 20: 19, 247: 19, 231: 19, 347: 19, 110: 19, 234: 18, 300: 18, 85: 18, 306: 18, 385: 18, 131: 18, 251: 17, 69: 17, 354: 17, 378: 16, 265: 16, 308: 16, 359: 16, 282: 16, 344: 16, 74: 16, 377: 16, 112: 16, 370: 15, 279: 15, 93: 15, 186: 14, 324: 14, 161: 14, 392: 14, 157: 14, 293: 14, 101: 14, 307: 14, 94: 14, 387: 14, 162: 13, 141: 13, 122: 13, 362: 13, 39: 13, 350: 13, 187: 13, 296: 13, 175: 13, 26: 13, 286: 13, 181: 13, 143: 13, 213: 12, 284: 12, 103: 12, 335: 12, 44: 12, 337: 12, 345: 12, 147: 12, 10: 12, 65: 12, 43: 12, 195: 12, 137: 11, 374: 11, 68: 11, 108: 11, 83: 11, 189: 11, 205: 11, 371: 11, 372: 11, 96: 11, 379: 11, 113: 11, 349: 10, 87: 10, 217: 10, 50: 10, 256: 10, 56: 10, 224: 10, 319: 10, 203: 10, 241: 10, 303: 10, 16: 10, 250: 10, 57: 10,

### Represent each document as list of its words vectors

In [ ]:
N=0 # 
min_doc= 0 # الحد الادني لعدد الكلمات في المستند

documents_embeddings = self_embedding_all_documents(data_DF , model,N)
labels =data_DF['class'].tolist()

print("the  len of labels and docs ", len(labels),len(documents_embeddings),"\n")

print("deleting docs that have not embedded words or its length less", min_doc , "\n")

#deleting empty document if exist
documents_embeddings ,labels = delete_empty_docs(documents_embeddings,labels,min_doc)
print("the  New len of labels and docs ", len(labels),len(documents_embeddings),"\n")

the  len of labels and docs  8491 8491 

deleting docs that have not embedded words or its length less 0 

[]
the  New len of labels and docs  8491 8491 



# Bag of weighted Concepts

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score


def data_preperation(dataset , labels, sample):
  x_train , x_test , y_train , y_test = train_test_split(dataset , labels , test_size = sample , random_state = 0)
  return array(x_train) , array(x_test) , array(y_train) , array(y_test)

def train_ML(x_train , y_train, krenell):
  #kernell = linear', 'poly', 'rbf', 'sigmoid',
  svm =  SVC(kernel=kernell, random_state=44).fit(x_train , y_train)   
  return svm

def evaluation_ML(x_test , y_test , svm_model):
  ev=svm_model.score(x_test , y_test)
  return ev

### Documents vectorization

In [ ]:
# Thershold similarity values
#Theta = [0.2,0.3,0.4,0.5,0.6]
Theta = [0.4]

number_of_concept_words = 5 # how many words to compare from each cluster
for alpha in Theta:
  
  min_idf = 0
  max_idf_r=1
  kernell='poly' 
  max_idf=max_idf_r* len(documents_embeddings)

  print("----------------------------------------------------------------------------------start represnation with similarity threshild = ", alpha,  ":\n")
  start_time = time.time()
  tf_similarity = BoWC_representation(documents_embeddings, number_of_concept_words , alpha , centers , sorted_cons)

  print("Counting IDF to select concepts with IDF less than ",max_idf, ":\n")

  idfcount,idfexp=calculate_IDF(tf_similarity)
  todelete=concept_to_delete(min_idf, max_idf,idfcount)
  print(len(todelete)," clusters to be deleted:  ", todelete ," \n")

  new_tf_similarity,idfexp,idfcount= clac_new_Tf_idf(todelete,tf_similarity,idfcount,idfexp)
  K_clusters=len(idfexp)
  print("The new number of clusters: ",K_clusters, ":\n")

  print("Calculating CF-IDF...\n")

  final_embeddings=CF_IDF(new_tf_similarity,idfexp)

  represnationtime=time.time() - start_time


  print("Time for representation: ", represnationtime," seconds")


  print("Start training the SVM model with ",kernell, " kernel ............................ ")


  embedding_DF = pd.DataFrame({'class': labels, 'document': final_embeddings})


  final_embeddings = [array(i).reshape(-1) for i in final_embeddings]
  labels =embedding_DF['class'].tolist()

  kernell='poly'

  x_train , x_test , y_train , y_test = data_preperation(final_embeddings , labels, 0.30)
  svm = train_ML(x_train , y_train,kernell)
  y_pred=svm.predict(x_test)
  F1 = f1_score(y_test,y_pred, average="micro")

  print("The micro average-weighted f1 score = ", F1)

----------------------------------------------------------------------------------start represnation with similarity threshild =  0.4 :

Counting IDF to select concepts with IDF less than  8491 :

0  clusters to be deleted:   []  

The new number of clusters:  400 :

Calculating CF-IDF...

Time for representation:  2398.9167873859406  seconds
Start training the SVM model with  poly  kernel ............................ 
The micro average-weighted f1 score =  0.9466248037676609
